In [1]:
from trading_helper import TradingHelper
import time
import math
app=TradingHelper()
mail="cirigliano.santiago@gmail.com"
portfolio=app.portfolio(mail=mail)
tickers_in_portfolio=[]
for i in portfolio["tickers"]:
    ticker=i["ticker"]
    if ticker not in ['AR$', 'US$ MEP','AL30','GD30','TVPP']:
        tickers_in_portfolio.append(ticker)

tickers_in_portfolio=tuple(tickers_in_portfolio)
print(tickers_in_portfolio)
while True:
    for ticker in tickers_in_portfolio: 
        try:stocks_available=app.stocks_available(mail=mail,ticker=ticker,term="CI",currency="pesos")
        except:stocks_available=app.stocks_available(mail=mail,ticker=ticker,term="CI",currency="pesos")
        if stocks_available["CI"]>0:
            amount_in_portfolio=stocks_available["CI"]
            try:
                response = app.get_snapshot(mail=mail, ticker=ticker)
            except Exception as e:
                # Reintentar mientras el error sea un 500
                while "Error code: 500" in str(e):
                    try:
                        response = app.get_snapshot(mail=mail, ticker=ticker)
                        break  # Salir del bucle si la solicitud es exitosa
                    except Exception as e:
                        if "Error code: 500" not in str(e):
                            print(f"Error al obtener snapshot: {e}")  # Registro del error diferente
                            break  # Salir del bucle si es otro error
                        continue  # Continúa reintentando si es un error 500
            ci = [item for item in response if item['currency'] == 'ARS' and item['term'] == 'CI'][0]
            hs =  [item for item in response if item['currency'] == 'ARS' and item['term'] == '24hs'][0]
            #print(f"CI {ci}")
            #print(f"hs {hs}")
            precio_venta=ci['bids'][0]['price']
            vol_venta=ci['bids'][0]['size']
            precio_compra=hs['asks'][0]['price']
            vol_compra=hs['asks'][0]['size']
            vol=min(vol_compra,vol_venta,amount_in_portfolio)
            if precio_venta == 0 or precio_compra == 0: break
            ratio=precio_venta/precio_compra
            #print(f"RATIO {ratio} con {ticker}. Volumen {vol} nominales. venta CI a ${precio_venta} y compra 24hs a ${precio_compra}")
            if ratio > 1.005 and ratio < 1.2:
                print(f"RATIO {ratio} con {ticker}. Volumen {vol} nominales. venta CI a ${precio_venta} y compra 24hs a ${precio_compra}")
                sell_order=app.place_sell_order(mail=mail,
                                                ticker=ticker,
                                                quantity=vol,
                                                price=precio_venta,
                                                term="CI",
                                                currency="pesos")
                print(f"Orden de venta {ticker} {sell_order}")
                order_status=app.check_order_status(mail=mail,
                                                    order_number=sell_order["Orden"])
                print(order_status)
                start_time = time.time()
                while order_status == "MARKET":
                    elapsed_time = time.time() - start_time
                    if elapsed_time > 2:
                        cancel_order=app.cancel_order(mail=mail,order_number=sell_order["Orden"])
                        print(f"Orden cancelada por Timeout: {cancel_order}")
                        break
                    order_status=app.check_order_status(mail=mail,
                                                    order_number=sell_order["Orden"])
                    print(f"Order Status de venta {order_status}")
                    
                if order_status == "EXECUTED":
                    buy_order=app.place_buy_order(mail=mail,
                                                ticker=ticker,
                                                quantity=vol,
                                                price=precio_compra,
                                                term="24",
                                                currency="pesos")
                    print(f"Orden de compra {ticker} {buy_order}")
                    order_status=app.check_order_status(mail=mail,
                                                        order_number=sell_order["Orden"])
                    while order_status == "MARKET":
                        order_status=app.check_order_status(mail=mail,
                                                    order_number=sell_order["Orden"])
                    print(f"Order status de Compra {order_status}")
                    if buy_order["Success"] == True: print("Negocio ATR")

('BPOD7', 'CTIO', 'PAMP', 'IRSA', 'TZX25', 'T2X5', 'CRES', 'S31E5', 'TX26', 'S28F5', 'T5X4', 'S13D4', 'TZV25')
RATIO 1.0333333333333334 con IRSA. Volumen 84 nominales. venta CI a $1550 y compra 24hs a $1500
Orden de venta IRSA {'Success': True, 'Orden': '4ae0d1d7-f82d-478e-bd9d-12988d76f5f0'}
EXECUTED
Orden de compra IRSA {'Success': True, 'Orden': '01f6e5ba-d44b-429d-ad58-e227e52f1bd1'}
Order status de Compra EXECUTED
Negocio ATR


ApiException: Bad HTTP API Response. Error code: 400. Server response {"error":"invalid_grant","error_description":"Invalid Refresh Token: Session Expired"}

In [30]:
import math
vol_venta = 100
vol_compra = 120
amount_in_portfolio = 80
precio_venta=155
precio_compra=149
minimum_vol = min(vol_compra,vol_venta,amount_in_portfolio)
if minimum_vol== vol_venta or minimum_vol==amount_in_portfolio:
    vol_compra = math.floor(vol_venta * (precio_venta/precio_compra))
elif minimum_vol==vol_compra:
    vol_venta = math.floor(vol_compra /(precio_venta/precio_compra))
print(vol_venta)
print(vol_compra)
#Esta mal el resultado cuando el minimo es lo que tengo en portfolio!!

"""
si el limite esta en lo que vendo en CI o lo que tengo en cartera:
papeles a comprar=papeles a vender * (redondeo(precio venta/precio compra)-1)
si el limite esta en lo que compro en 24hs
papeles a vender=papeles a comprar/(redondeo(precio venta/precio compra)-1)"""

100
104


'\nsi el limite esta en lo que vendo en CI o lo que tengo en cartera:\npapeles a comprar=papeles a vender * (redondeo(precio venta/precio compra)-1)\nsi el limite esta en lo que compro en 24hs\npapeles a vender=papeles a comprar/(redondeo(precio venta/precio compra)-1)'

In [ ]:
from trading_helper import TradingHelper
app=TradingHelper()
portfolio=app.portfolio(mail="cirigliano.santiago@gmail.com")
tickers_in_portfolio=[]
for i in portfolio["tickers"]:
    ticker=i["ticker"]
    tickers_in_portfolio.append(ticker)
tickers_in_portfolio=tuple(tickers_in_portfolio[2:])
print(tickers_in_portfolio)

('BPOD7', 'AL30', 'CTIO', 'PAMP', 'TZX25', 'IRSA', 'T2X5', 'CRES', 'S31E5', 'TX26', 'S28F5', 'T5X4', 'S13D4', 'TZV25')


In [8]:
import pandas as pd
df=pd.read_csv("tabla_tickers.csv")
df.sample(5)

,short_ticker,tipo,vencimiento
39,PAMP,accion,NaN
16,TZXD7,cer,15/12/2027
9,TZX27,cer,30/06/2027
6,TX28,cer,9/11/2028
38,TXAR,accion,NaN
